In [56]:
# Import needed modules
import pandas as pd # type: ignore

In [57]:
# Load the data
path_csv = "./blastn_neg_noMatchCds_plus100nt.csv"
df = pd.read_csv(path_csv, sep=",", header=0)
# df.reset_index(drop=True, inplace=True)
print(df.shape)
df.head()
print(df.dtypes)
df.head()

(2327, 14)
qseqid       object
sseqid       object
pident      float64
length        int64
mismatch      int64
gapopen       int64
qstart        int64
qend          int64
sstart        int64
send          int64
evalue      float64
bitscore    float64
qlen          int64
slen          int64
dtype: object


,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,qlen,slen
0,Seq_0_LinJ.01,Seq_0_LinJ.01,100.0,140,0,0,1,140,1,140,2.720000e-71,259.0,140,140
1,Seq_1_LinJ.01,Seq_1_LinJ.01,100.0,629,0,0,1,629,1,629,0.000000e+00,1162.0,629,629
2,Seq_2_LinJ.01,Seq_2_LinJ.01,100.0,119,0,0,1,119,1,119,1.060000e-59,220.0,119,119
3,Seq_3_LinJ.01,Seq_3_LinJ.01,100.0,574,0,0,1,574,1,574,0.000000e+00,1061.0,574,574
4,Seq_4_LinJ.01,Seq_4_LinJ.01,100.0,288,0,0,1,288,1,288,3.180000e-153,532.0,288,288


In [58]:
# Let's see the cunt values in the "qseqid"
df["qseqid"].value_counts()

qseqid
Seq_266_LinJ.22    368
Seq_5_LinJ.01      127
Seq_265_LinJ.22    100
Seq_560_LinJ.34     15
Seq_566_LinJ.34     15
                  ... 
Seq_409_LinJ.30      1
Seq_407_LinJ.30      1
Seq_343_LinJ.27      1
Seq_336_LinJ.27      1
Seq_390_LinJ.29      1
Name: count, Length: 721, dtype: int64

In [59]:
# Since there are some elements that have only one coun value, which is for sure with himself, let's remove the rows where "qseqid" and "sseqid" are the same
df = df[df["qseqid"] != df["sseqid"]]
print(df.shape)
df.head()

(1410, 14)


,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,qlen,slen
24,Seq_5_LinJ.01,Seq_266_LinJ.22,94.069,2664,76,58,412,3025,1,2632,0.0,3969.0,3406,24324
25,Seq_5_LinJ.01,Seq_266_LinJ.22,93.548,2666,92,60,424,3045,557,3186,0.0,3897.0,3406,24324
26,Seq_5_LinJ.01,Seq_266_LinJ.22,93.580,2648,94,54,414,3025,17616,20223,0.0,3879.0,3406,24324
27,Seq_5_LinJ.01,Seq_266_LinJ.22,93.594,2638,96,49,424,3025,21350,23950,0.0,3868.0,3406,24324
28,Seq_5_LinJ.01,Seq_266_LinJ.22,93.530,2643,95,51,424,3025,20816,23423,0.0,3864.0,3406,24324


In [60]:
# Let's count values again
df["qseqid"].value_counts()

qseqid
Seq_266_LinJ.22    189
Seq_5_LinJ.01      108
Seq_265_LinJ.22     99
Seq_560_LinJ.34     14
Seq_566_LinJ.34     14
                  ... 
Seq_660_LinJ.35      1
Seq_661_LinJ.35      1
Seq_663_LinJ.35      1
Seq_664_LinJ.35      1
Seq_665_LinJ.35      1
Name: count, Length: 341, dtype: int64

In [69]:
# Let's check if there are values in "sstart" > "send"
print(sum(df["sstart"] > df["send"]))
df[df["sstart"] > df["send"]].head()

68


,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,qlen,slen
154,Seq_20_LinJ.04,Seq_21_LinJ.04,99.281,278,2,0,10,287,282,5,6.170000e-144,503.0,690,676
155,Seq_20_LinJ.04,Seq_24_LinJ.04,87.019,208,13,3,97,290,222,15,1.940000e-59,222.0,690,222
157,Seq_21_LinJ.04,Seq_20_LinJ.04,99.281,278,2,0,5,282,287,10,6.040000e-144,503.0,676,690
163,Seq_24_LinJ.04,Seq_20_LinJ.04,87.624,202,11,3,21,222,284,97,5.900000e-60,222.0,222,690
176,Seq_34_LinJ.05,Seq_35_LinJ.05,98.039,357,5,1,13,367,357,1,5.870000e-179,619.0,689,685


In [73]:
# Change the values so "sstart" is always < "send"
df.loc[
    df["sstart"] > df["send"],
    ["sstart", "send"]
] = df.loc[
    df["sstart"] > df["send"],
    ["send", "sstart"]
].values

# Let's check if there are values in "sstart" > "send"
print(sum(df["sstart"] > df["send"]))
df[df["sstart"] > df["send"]].head()

0


,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,qlen,slen


In [74]:
# Let's make a "groupby" object
grouped_df = df.groupby("sseqid")
print(f"grouped_df has {len(grouped_df)} groups")

grouped_df has 341 groups


In [76]:
group_test = pd.DataFrame()
stop = 7
for index,(group_name, group_data) in enumerate(grouped_df, start=1):
    # print(f"index: {index}; group: {group_name}")
    min_sstart = group_data["sstart"].min()
    max_send = group_data["send"].max()
    slen = group_data["slen"].max()
    print("="*50)
    print(f"For group {group_name} the min_sstart is {min_sstart}, the max_send is {max_send} and the slen is {slen}")

print(group_test.shape)
group_test.head()

For group Seq_100_LinJ.09 the min_sstart is 1, the max_send is 309 and the slen is 333
For group Seq_104_LinJ.10 the min_sstart is 1, the max_send is 795 and the slen is 1009
For group Seq_119_LinJ.12 the min_sstart is 1, the max_send is 256 and the slen is 256
For group Seq_120_LinJ.12 the min_sstart is 1, the max_send is 523 and the slen is 607
For group Seq_121_LinJ.12 the min_sstart is 1, the max_send is 258 and the slen is 258
For group Seq_122_LinJ.12 the min_sstart is 1, the max_send is 523 and the slen is 607
For group Seq_124_LinJ.12 the min_sstart is 1, the max_send is 531 and the slen is 531
For group Seq_126_LinJ.12 the min_sstart is 1, the max_send is 724 and the slen is 724
For group Seq_127_LinJ.12 the min_sstart is 1, the max_send is 725 and the slen is 725
For group Seq_128_LinJ.12 the min_sstart is 2, the max_send is 750 and the slen is 1424
For group Seq_12_LinJ.02 the min_sstart is 1, the max_send is 323 and the slen is 323
For group Seq_130_LinJ.12 the min_sstart i

""
